# PRACTICA 3

**Nombre:** Jorge Osvaldo Gonzalez Gonzalez
**e-mail:** jorge.gonzalez1611@alumnos.udg.mx

## MODULES

In [1]:
import math
import os 

import numpy as np
import pandas as pd
import panel as pn
import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance


import holoviews as hv
from panel.template import DarkTheme

pn.extension()


## CLASES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

### Provided

In [3]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

### Own

#### Parsers

In [4]:
def np_arrays_to_df(arrays, column_names): 
    df_res = pd.DataFrame(columns=column_names)
    for i in range(0, len(arrays[0])):
        df_res = pd.concat([df_res, pd.DataFrame(columns=column_names, data=[[arrays[j][i]  for j in range(0, len(column_names))]])], ignore_index=True)
    return df_res

#### Files

In [5]:
def save_df_to_csv(df_in, path, name):
    os.makedirs(path, exist_ok=True)
    df_in.to_csv(f'{path}/{name}.csv', index=False)

#### Generators

###### Brownian Motion

In [6]:
def generate_bm(steps, speed, start_x, start_y):
    trajectory = Vec2d([start_x + speed, start_y])
    CARDINAL_POINTS = [np.pi * 0.5, 0, np.pi, np.pi * 1.5]
    BM_2d = np.zeros(shape=(steps, 2))

    # Generate trajectory by accumulation of displacements4
    for i in range(1, steps):
        turn_angle = np.random.choice(CARDINAL_POINTS)
        trajectory = trajectory.rotated(turn_angle)
        BM_2d[i, 0] = BM_2d[i - 1, 0] + trajectory.x
        BM_2d[i, 1] = BM_2d[i - 1, 1] + trajectory.y

    return np_arrays_to_df(BM_2d, ['x_pos', 'y_pos'])


##### Correlated Random Walk

In [7]:
def generate_crw(steps, speed, start_x, start_y, coef):
    cur_pos = Vec2d(start_x + speed, start_y)
    crw_df = pd.DataFrame([{'x_pos': 0, 'y_pos': 0}])
    crw_turns = wrapcauchy.rvs(c=coef, loc=0, size=steps-1)
    for i in range(1, steps):
        cur_pos = cur_pos.rotated(crw_turns[i-1])
        crw_df = pd.concat([crw_df, pd.DataFrame([{'x_pos': crw_df.iloc[i-1, 0] + cur_pos.x, 'y_pos': crw_df.iloc[i-1, 1] + cur_pos.y}])]) 
    return crw_df

##### Levy Flight

In [8]:
def generate_levy(steps, speed, start_x, start_y, cauchy, alpha):
    cur_pos_lw = Vec2d(start_x + speed, start_y)
    lw_2d_df = pd.DataFrame([{"x_pos": 0, "y_pos": 0}])
    # Generate trajectory by accumulation of displacements4
    i = 1
    while i <= steps:
        turn_angle = wrapcauchy.rvs(c=cauchy, loc=0)
        cur_pos_lw = cur_pos_lw.rotated(turn_angle)
        cur_steps = abs(round(levy_stable.rvs(alpha=alpha, beta=1, loc=3))) 

        for j in range(0, int(cur_steps)):
            lw_2d_df = pd.concat([lw_2d_df, pd.DataFrame([{'x_pos': lw_2d_df.x[i-1]+cur_pos_lw.x, 'y_pos': lw_2d_df.y[i-1]+cur_pos_lw.y}])], ignore_index=True)
            i+=1
            if i > steps:
                break
    return lw_2d_df

#### METRICS

##### Path Length

In [9]:
def path_length(df):
    dis_df = pd.DataFrame([{"dis": 0}])
    for i in range(1, df.shape[0]):
        pointA = np.array((df.iloc[i-1].x_pos, df.iloc[i-1].y_pos)) 
        pointB = np.array((df.iloc[i].x_pos, df.iloc[i].y_pos))
        dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
        dis_df = pd.concat([dis_df, pd.DataFrame([{"dis": dis_df.iloc[i-1].dis + dis}])], ignore_index=True)
    return np_arrays_to_df(dis_df, ["dis"])

###### MSD

In [10]:
###################################################################################
# MSD VECTOR
###################################################################################
def MSD(df_in):
    displacement_vector = np.zeros(df_in.shape[0] - 1)
    for tau in range(1,df_in.shape[0]):
        ## start - Add your code here
        MSD = np.array([distance.euclidean(df_in.iloc[i-tau], df_in.iloc[i]) ** 2
            for i in range(tau, df_in.shape[0], 1)])
        displacement_vector[tau-1] = MSD.mean()
    return np_arrays_to_df(displacement_vector, ["dis"])

## IMPLEMENTATION

### Data Frame

In [11]:
df_path = pd.DataFrame(columns=["x_pos", "y_pos"])
df_metric = pd.DataFrame(columns=["dis"])

### Layout Definition

In [12]:
bootstrap = pn.template.MaterialTemplate(title='Tópico de Industria I', site='Tarea 3', header_background='#000000', theme=DarkTheme)

kindOf = pn.widgets.RadioButtonGroup(name='Tipo de Trayectoria', options=['BM', 'CRW', 'Levy'], button_type='primary', )
steps = pn.widgets.IntSlider(name="Pasos", start=100, end = 1000, value=100)
speed = pn.widgets.FloatSlider(name="Velocidad", start=0.1, end=20, value=1, step=0.1)
start_x = pn.widgets.IntInput(name="x", start=0, value=1, end=10000, width=75)
start_y = pn.widgets.IntInput(name="y", start=0, value=0, end=10000, width=75)
row_pos = pn.Row(start_x, start_y)
cauchy_coef = pn.widgets.FloatSlider(name="Coeficiente Cauchy",value=0.01, start=0.01, end=.99, step=0.01)
levy_alpha = pn.widgets.FloatSlider(name="Levy Alpha", value=0.01, start=0.01, end=2, step=0.01)
metric_kind = pn.widgets.RadioButtonGroup(name="Tipo de Métrica", options=['Path Length' , 'Mean Square Displacement'], button_type='warning')

bootstrap.sidebar.append(pn.widgets.StaticText(value="Tipo de Trayectoria"))
bootstrap.sidebar.append(kindOf)
bootstrap.sidebar.append(steps)
bootstrap.sidebar.append(speed)
bootstrap.sidebar.append(pn.widgets.StaticText(value="Posición"))
bootstrap.sidebar.append(row_pos)
bootstrap.sidebar.append(cauchy_coef)
bootstrap.sidebar.append(levy_alpha)
bootstrap.sidebar.append(pn.widgets.StaticText(value="Métrica a evaluar"))
bootstrap.sidebar.append(metric_kind)
levy_alpha.visible = False
cauchy_coef.visible = False

card_fig_path = pn.Card(title='Ruta')
card_fig_met = pn.Card(title='Métrica')

row_fig = pn.Row(card_fig_path, card_fig_met)
bootstrap.main.append(row_fig)


### Events


In [17]:

def recalculate():
    if kindOf.value == 'BM':
        df_path = generate_bm(steps.value, speed.value, start_x.value, start_y.value)
    elif kindOf.value == 'CRW':
        df_path = generate_crw(steps.value, speed.value, start_x.value, start_y.value, cauchy_coef.value)
    elif kindOf.value == 'Levy':
        df_path = generate_levy(steps.value, speed.value, start_x.value, start_y.value, cauchy_coef.value, levy_alpha.value)
    if metric_kind.value == 'Path Length':
        df_metric = path_length(df_path)
    elif metric_kind.value == 'Mean Square Displacement':
        df_metric = MSD(df_path)

def kind_change(*events):
    if kindOf.value=='CRW' or kindOf.value =='Levy':
        cauchy_coef.visible = True
        if kindOf.value == 'Levy':
            levy_alpha.visible = True 
        else:
            levy_alpha.visible = False
            levy_alpha.value = 0.1
    else:
        cauchy_coef.visible = False
        cauchy_coef.value = 0.1
    recalculate()

def values_change(*events):
    recalculate()
def steps_change(*events):
    recalculate()
def speed_change(*events):
    recalculate()
def sx_change(*events):
    recalculate()
def sy_change(*events):
    recalculate()
def cauchy_change(*events):
    recalculate()
def levy_change(*events):
    recalculate()
def metric_change(*events):
    recalculate()

kindOf.param.watch(kind_change, ['value'], onlychanged=False)
steps.param.watch(steps_change, ['value'], onlychanged=False)
speed.param.watch(speed_change, ['value'], onlychanged=False)
start_x.param.watch(sx_change, ['value'], onlychanged=False)
start_y.param.watch(sy_change, ['value'], onlychanged=False)
cauchy_coef.param.watch(cauchy_change, ['value'], onlychanged=False)
levy_alpha.param.watch(levy_change, ['value'], onlychanged=False)
metric_kind.param.watch(metric_change, ['value'], onlychanged=False)

    


Watcher(inst=RadioButtonGroup(button_type='warning', name='Tipo de Métrica', options=['Path Length', ...], value='Path Length'), cls=<class 'panel.widgets.select.RadioButtonGroup'>, fn=<function metric_change at 0x00000207C0658670>, mode='args', onlychanged=False, parameter_names=('value',), what='value', queued=False, precedence=0)

### Layout Show

In [ ]:
bootstrap.show()